<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/annotation_import/image.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>




<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/annotation_import/image.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Image annotation import
This notebook will provide examples of each supported annotation type for image assets. 

### [Model-assisted labeling (MAL)](https://docs.labelbox.com/docs/model-assisted-labeling)

* This workflow allows you to import computer-generated predictions (or simply annotations created outside of Labelbox) as pre-labels on an asset. 

The imported annotations will be pre-populated in the labeling editor. However, in order to convert the pre-labels to real annotations, a human labeler will still need to open the Data Row in the Editor and submit it. This functionality is designed to speed up human labeling.

### [Import ground truth](https://docs.labelbox.com/docs/import-ground-truth)

* This  workflow functionality allows you to bulk import your ground truth annotations from an external or third-party labeling system into Labelbox Annotate. Using the label import API to import external data is a useful way to consolidate and migrate all annotations into Labelbox as a single source of truth.

### Python annotation types vs NDJSON
**Python annotation type (recommended)**
- Provides a seamless transition between third-party platforms, machine learning pipelines, and Labelbox.

- Allows you to build annotations locally with local file paths, numpy arrays, or URLs

- Easily convert Python Annotation Type format to NDJSON format to quickly import annotations to Labelbox

- It supports one-level nested classification (free text / radio / checklist) under the object or classification annotation.

**NDJSON**
- Skip formatting annotation payload in the Python Annotation Types format just to convert back to NDJSON

- Ability to create the payload in the NDJSON import format directly

- It supports any levels of nested classification (free text / radio / checklist) under the object or classification annotation.

## Setup

In [ ]:
!pip install -q "labelbox[data]"

In [ ]:
import uuid
import requests
import labelbox as lb
import labelbox.types as lb_types

## Replace with your API key

Guides on [Create an API key](https://docs.labelbox.com/docs/create-an-api-key)

In [ ]:
API_KEY = ""
client = lb.Client(API_KEY)

## Supported annotations for image


### Classification : Radio (single-choice)

In [ ]:
# Python annotation
radio_annotation = lb_types.ClassificationAnnotation(
    name="radio_question",
    value=lb_types.Radio(answer=lb_types.ClassificationAnswer(
        name="second_radio_answer")))

# NDJSON
radio_annotation_ndjson = {
    "name": "radio_question",
    "answer": {
        "name": "second_radio_answer"
    }
}

### Classification: Checklist (multi-choice)

In [ ]:
# Python annotation
checklist_annotation = lb_types.ClassificationAnnotation(
    name="checklist_question",  # must match your ontology feature"s name
    value=lb_types.Checklist(answer=[
        lb_types.ClassificationAnswer(name="first_checklist_answer"),
        lb_types.ClassificationAnswer(name="second_checklist_answer")
    ]))

# NDJSON
checklist_annotation_ndjson = {
    "name": "checklist_question",
    "answer": [{
        "name": "first_checklist_answer"
    }, {
        "name": "second_checklist_answer"
    }]
}

### Classification: Nested radio and checklist



In [ ]:
nested_radio_annotation = lb_types.ClassificationAnnotation(
  name="nested_radio_question",
  value=lb_types.Radio(
    answer=lb_types.ClassificationAnswer(
      name="first_radio_answer",
      classifications=[
        lb_types.ClassificationAnnotation(
          name="sub_radio_question",
          value=lb_types.Radio(
            answer=lb_types.ClassificationAnswer(
              name="first_sub_radio_answer"
            )
          )
        )
      ]
    )
  )
)
# NDJSON
nested_radio_annotation_ndjson = {
    "name": "nested_radio_question",
    "answer": {
        "name": "first_radio_answer",
          "classifications": [{
              "name": "sub_radio_question",
              "answer": {
                  "name": "first_sub_radio_answer"
              }
          }]
    }
}


nested_checklist_annotation = lb_types.ClassificationAnnotation(
  name="nested_checklist_question",
  value=lb_types.Checklist(
    answer=[lb_types.ClassificationAnswer(
      name="first_checklist_answer",
      classifications=[
        lb_types.ClassificationAnnotation(
          name="sub_checklist_question",
          value=lb_types.Checklist(
            answer=[lb_types.ClassificationAnswer(
            name="first_sub_checklist_answer"
          )]
        ))
      ]
    )]
  )
)

nested_checklist_annotation_ndjson = {
    "name": "nested_checklist_question",
    "answer": [{
        "name": "first_checklist_answer",
        "classifications": [{
            "name": "sub_checklist_question",
            "answer": {
                "name": "first_sub_checklist_answer"
            }
        }]
    }]
}

### Classification: Free-form text 

In [ ]:
# Python annotation
text_annotation = lb_types.ClassificationAnnotation(
    name="free_text",  # must match your ontology feature"s name
    value=lb_types.Text(answer="sample text"))

# NDJSON
text_annotation_ndjson = {
    "name": "free_text",
    "answer": "sample text",
}

### Relationship with bounding box
> **NOTE:**  
> Only supported for MAL imports

In [ ]:
# Python Annotation
bbox_source = lb_types.ObjectAnnotation(
    name="bounding_box",
    value=lb_types.Rectangle(
        start=lb_types.Point(x=2096, y=1264),
        end=lb_types.Point(x=2240, y=1689),
    ),
)

bbox_target = lb_types.ObjectAnnotation(
    name="bounding_box",
    value=lb_types.Rectangle(
        start=lb_types.Point(x=2272, y=1346),
        end=lb_types.Point(x=2416, y=1704),
    ),
)

relationship = lb_types.RelationshipAnnotation(
    name="relationship",
    value=lb_types.Relationship(
        source=bbox_source,
        target=bbox_target,
        type=lb_types.Relationship.Type.UNIDIRECTIONAL,
    ))

## Only supported for MAL imports
uuid_source = str(uuid.uuid4())
uuid_target = str(uuid.uuid4())

bbox_source_ndjson = {
    "uuid": uuid_source,
    "name": "bounding_box",
    "bbox": {
        "top": 1264.0,
        "left": 2096.0,
        "height": 425.0,
        "width": 144.0
    }
}

bbox_target_ndjson = {
    "uuid": uuid_target,
    "name": "bounding_box",
    "bbox": {
        "top": 1346.0,
        "left": 2272.0,
        "height": 358.0,
        "width": 144.0
    }
}

relationship_ndjson = {
    "name": "relationship",
    "relationship": {
        "source": uuid_source,
        "target": uuid_target,
        "type": "unidirectional"
    }
}

### Bounding box

In [ ]:
# Python annotation
bbox_annotation = lb_types.ObjectAnnotation(
    name="bounding_box",  # must match your ontology feature"s name
    value=lb_types.Rectangle(
        start=lb_types.Point(x=1690, y=977),  #  x = left, y = top
        end=lb_types.Point(x=1915, y=1307),  # x= left + width , y = top + height
    ))

# NDJSON
bbox_annotation_ndjson = {
    "name": "bounding_box",
    "bbox": {
        "top": 977,
        "left": 1690,
        "height": 330,
        "width": 225
    }
}

### Bounding box with nested classification

In [ ]:
# Python annotation
bbox_with_radio_subclass_annotation = lb_types.ObjectAnnotation(
    name="bbox_with_radio_subclass",
    value=lb_types.Rectangle(
        start=lb_types.Point(x=541, y=933),  #  x = left, y = top
        end=lb_types.Point(x=871, y=1124),  # x= left + width , y = top + height
    ),
    classifications=[
        lb_types.ClassificationAnnotation(
            name="sub_radio_question",
            value=lb_types.Radio(answer=lb_types.ClassificationAnswer(
                name="first_sub_radio_answer")))
    ])

## NDJSON
bbox_with_radio_subclass_ndjson = {
    "name": "bbox_with_radio_subclass",
    "classifications": [{
        "name": "sub_radio_question",
        "answer": {
            "name": "first_sub_radio_answer"
        }
    }],
    "bbox": {
        "top": 933,
        "left": 541,
        "height": 191,
        "width": 330
    }
}

### Polygon

In [ ]:
# Python annotation
polygon_annotation = lb_types.ObjectAnnotation(
    name="polygon",  # must match your ontology feature"s name
    value=lb_types.Polygon(  # Coordinates for the vertices of your polygon
        points=[
            lb_types.Point(x=1489.581, y=183.934),
            lb_types.Point(x=2278.306, y=256.885),
            lb_types.Point(x=2428.197, y=200.437),
            lb_types.Point(x=2560.0, y=335.419),
            lb_types.Point(x=2557.386, y=503.165),
            lb_types.Point(x=2320.596, y=503.103),
            lb_types.Point(x=2156.083, y=628.943),
            lb_types.Point(x=2161.111, y=785.519),
            lb_types.Point(x=2002.115, y=894.647),
            lb_types.Point(x=1838.456, y=877.874),
            lb_types.Point(x=1436.53, y=874.636),
            lb_types.Point(x=1411.403, y=758.579),
            lb_types.Point(x=1353.853, y=751.74),
            lb_types.Point(x=1345.264, y=453.461),
            lb_types.Point(x=1426.011, y=421.129)
        ]))

# NDJSON
polygon_annotation_ndjson = {
  "name": "polygon",
  "polygon": [
    {"x": 1489.581, "y": 183.934},
    {"x": 2278.306, "y": 256.885},
    {"x": 2428.197, "y": 200.437},
    {"x": 2560.0, "y": 335.419},
    {"x": 2557.386, "y": 503.165},
    {"x": 2320.596, "y": 503.103},
    {"x": 2156.083, "y": 628.943},
    {"x": 2161.111, "y": 785.519},
    {"x": 2002.115, "y": 894.647},
    {"x": 1838.456, "y": 877.874},
    {"x": 1436.53, "y": 874.636},
    {"x": 1411.403, "y": 758.579},
    {"x": 1353.853, "y": 751.74},
    {"x": 1345.264, "y": 453.461},
    {"x": 1426.011, "y": 421.129},
    {"x": 1489.581, "y": 183.934}
  ]
}

### Segmentation Mask

In [ ]:
### Raster Segmentation (Byte string array)
url = "https://storage.googleapis.com/labelbox-datasets/image_sample_data/raster_seg.png"
response = requests.get(url)

mask_data = lb.types.MaskData(im_bytes=response.content) # You can also use "url" instead of img_bytes to pass the PNG mask url.
mask_annotation = lb_types.ObjectAnnotation(
    name="mask",
    value=lb_types.Mask(
        mask=mask_data,
        color=(255, 255, 255))
)

# NDJSON using instanceURI, bytes array is not fully supported.
mask_annotation_ndjson = {
  "name": "mask",
  "classifications": [],
  "mask": {"instanceURI": url,
  "colorRGB": (255, 255, 255)}
}


### Segmentation mask with nested classification 

In [ ]:
url_2 = "https://storage.googleapis.com/labelbox-datasets/image_sample_data/raster_seg_with_subclass.png"
response = requests.get(url_2)
mask_data = lb_types.MaskData(im_bytes=response.content)

# Python annotation
mask_with_text_subclass_annotation = lb_types.ObjectAnnotation(
  name = "mask_with_text_subclass", # must match your ontology feature"s name
  value=lb_types.Mask(
    mask=mask_data,
    color=(255, 255, 255)),
  classifications=[
    lb_types.ClassificationAnnotation(
      name="sub_free_text",
      value=lb_types.Text(answer="free text answer")
    )]
)

# NDJSON using instanceURI, bytes array is not fully supported.
mask_with_text_subclass_ndjson = {
  "name": "mask_with_text_subclass",
  "mask": {"instanceURI": url_2,
  "colorRGB": (255, 255, 255)},
  "classifications":[{
    "name": "sub_free_text",
    "answer": "free text answer"
  }]
}

### Point

In [ ]:
# Python annotation
point_annotation = lb_types.ObjectAnnotation(
    name="point",  # must match your ontology feature"s name
    value=lb_types.Point(x=1166.606, y=1441.768),
)

# NDJSON
point_annotation_ndjson = {
    "name": "point",
    "classifications": [],
    "point": {
        "x": 1166.606,
        "y": 1441.768
    }
}

### Polyline

In [ ]:
# Python annotation
polyline_annotation = lb_types.ObjectAnnotation(
    name="polyline",  # must match your ontology feature"s name
    value=lb_types.Line(  # Coordinates for the keypoints in your polyline
        points=[
            lb_types.Point(x=2534.353, y=249.471),
            lb_types.Point(x=2429.492, y=182.092),
            lb_types.Point(x=2294.322, y=221.962),
            lb_types.Point(x=2224.491, y=180.463),
            lb_types.Point(x=2136.123, y=204.716),
            lb_types.Point(x=1712.247, y=173.949),
            lb_types.Point(x=1703.838, y=84.438),
            lb_types.Point(x=1579.772, y=82.61),
            lb_types.Point(x=1583.442, y=167.552),
            lb_types.Point(x=1478.869, y=164.903),
            lb_types.Point(x=1418.941, y=318.149),
            lb_types.Point(x=1243.128, y=400.815),
            lb_types.Point(x=1022.067, y=319.007),
            lb_types.Point(x=892.367, y=379.216),
            lb_types.Point(x=670.273, y=364.408),
            lb_types.Point(x=613.114, y=288.16),
            lb_types.Point(x=377.559, y=238.251),
            lb_types.Point(x=368.087, y=185.064),
            lb_types.Point(x=246.557, y=167.286),
            lb_types.Point(x=236.648, y=285.61),
            lb_types.Point(x=90.929, y=326.412)
        ]),
)

# NDJSON
polyline_annotation_ndjson = {
  "name": "polyline",
  "classifications": [],
  "line": [
    {"x": 2534.353, "y": 249.471},
    {"x": 2429.492, "y": 182.092},
    {"x": 2294.322, "y": 221.962},
    {"x": 2224.491, "y": 180.463},
    {"x": 2136.123, "y": 204.716},
    {"x": 1712.247, "y": 173.949},
    {"x": 1703.838, "y": 84.438},
    {"x": 1579.772, "y": 82.61},
    {"x": 1583.442, "y": 167.552},
    {"x": 1478.869, "y": 164.903},
    {"x": 1418.941, "y": 318.149},
    {"x": 1243.128, "y": 400.815},
    {"x": 1022.067, "y": 319.007},
    {"x": 892.367, "y": 379.216},
    {"x": 670.273, "y": 364.408},
    {"x": 613.114, "y": 288.16},
    {"x": 377.559, "y": 238.251},
    {"x": 368.087, "y": 185.064},
    {"x": 246.557, "y": 167.286},
    {"x": 236.648, "y": 285.61},
    {"x": 90.929, "y": 326.412}
  ]
}

# End-to-end example: Import pre-labels or ground truth

## Step 1: Import data rows into catalog



In [ ]:
# send a sample image as batch to the project
global_key = "2560px-Kitano_Street_Kobe01s5s41102.jpeg"

test_img_url = {
    "row_data":
        "https://storage.googleapis.com/labelbox-datasets/image_sample_data/2560px-Kitano_Street_Kobe01s5s4110.jpeg",
    "global_key":
        global_key
}

dataset = client.create_dataset(name="image-demo-dataset")
task = dataset.create_data_rows([test_img_url])
task.wait_till_done()

print(f"Errors: {task.errors}")
print(f"Failed data rows: {task.failed_data_rows}")

## Step 2: Create/select an ontology
Your project should have the correct ontology setup with all the tools and classifications supported for your annotations and the tool names and classification instructions should match the `name` fields in your annotations to ensure the correct feature schemas are matched.

For example, when we created the bounding box annotation above, we provided the `name` as `bounding_box`. Now, when we setup our ontology, we must ensure that the name of the bounding box tool is also `bounding_box`. The same alignment must hold true for the other tools and classifications we create in our ontology.

In [ ]:
ontology_builder = lb.OntologyBuilder(
    classifications=[  # List of Classification objects
        lb.Classification(class_type=lb.Classification.Type.RADIO,
                          name="radio_question",
                          options=[
                              lb.Option(value="first_radio_answer"),
                              lb.Option(value="second_radio_answer")
                          ]),
        lb.Classification(class_type=lb.Classification.Type.CHECKLIST,
                          name="checklist_question",
                          options=[
                              lb.Option(value="first_checklist_answer"),
                              lb.Option(value="second_checklist_answer")
                          ]),
        lb.Classification(class_type=lb.Classification.Type.TEXT,
                          name="free_text"),
        lb.Classification(
            class_type=lb.Classification.Type.RADIO,
            name="nested_radio_question",
            options=[
                lb.Option("first_radio_answer",
                          options=[
                              lb.Classification(
                                  class_type=lb.Classification.Type.RADIO,
                                  name="sub_radio_question",
                                  options=[lb.Option("first_sub_radio_answer")])
                          ])
            ]),
        lb.Classification(
            class_type=lb.Classification.Type.CHECKLIST,
            name="nested_checklist_question",
            options=[
                lb.Option(
                    "first_checklist_answer",
                    options=[
                        lb.Classification(
                            class_type=lb.Classification.Type.CHECKLIST,
                            name="sub_checklist_question",
                            options=[lb.Option("first_sub_checklist_answer")])
                    ])
            ]),
    ],
    tools=[  # List of Tool objects
        lb.Tool(tool=lb.Tool.Type.BBOX, name="bounding_box"),
        lb.Tool(tool=lb.Tool.Type.BBOX,
                name="bbox_with_radio_subclass",
                classifications=[
                    lb.Classification(
                        class_type=lb.Classification.Type.RADIO,
                        name="sub_radio_question",
                        options=[lb.Option(value="first_sub_radio_answer")]),
                ]),
        lb.Tool(tool=lb.Tool.Type.POLYGON, name="polygon"),
        lb.Tool(tool=lb.Tool.Type.RASTER_SEGMENTATION, name="mask"),
        lb.Tool(tool=lb.Tool.Type.RASTER_SEGMENTATION,
                name="mask_with_text_subclass",
                classifications=[
                    lb.Classification(
                        class_type=lb.Classification.Type.TEXT,
                        name="sub_free_text")
                    ]
                ),
        lb.Tool(tool=lb.Tool.Type.POINT, name="point"),
        lb.Tool(tool=lb.Tool.Type.LINE, name="polyline"),
        lb.Tool(tool=lb.Tool.Type.RELATIONSHIP, name="relationship")
    ])

ontology = client.create_ontology("Image Annotation Import Demo Ontology",
                                  ontology_builder.asdict(),
                                  media_type=lb.MediaType.Image
                                  )

## Step 3: Create a labeling project
Connect the ontology to the labeling project

In [ ]:
# Project defaults to batch mode with benchmark quality settings if this argument is not provided
# Queue mode will be deprecated once dataset mode is deprecated
project = client.create_project(name="Image Annotation Import Demo",
                                media_type=lb.MediaType.Image)

project.setup_editor(ontology)

## Step 4: Send a batch of data rows to the project


In [ ]:
batch = project.create_batch(
    "image-demo-batch",  # each batch in a project must have a unique name
    global_keys=[
        global_key
    ],  # paginated collection of data row objects, list of data row ids or global keys
    priority=1  # priority between 1(highest) - 5(lowest)
)

print(f"Batch: {batch}")

## Step 5: Create the annotations payload

Create the annotations payload using the snippets of code above

Labelbox support two formats for the annotations payload: NDJSON and Python Annotation types. Both are described below. If you are using Python Annotation types, compose your annotations into Labels attached to the data rows.

### Python annotations

Here we create the complete label ndjson payload of annotations only using python annotation format. There is one annotation for each reference to an annotation that we created.

In [ ]:
label = []
annotations = [
    radio_annotation,
    nested_radio_annotation,
    checklist_annotation,
    nested_checklist_annotation,
    text_annotation,
    bbox_annotation,
    bbox_with_radio_subclass_annotation,
    polygon_annotation,
    mask_annotation,
    mask_with_text_subclass_annotation,
    point_annotation,
    polyline_annotation,
    bbox_source,
    bbox_target,
    relationship,
]
label.append(
    lb_types.Label(data=lb_types.ImageData(global_key=global_key),
                   annotations=annotations))

### NDJSON annotations
Here we create the complete label ndjson payload of annotations only using NDJSON format. There is one annotation for each reference to an annotation that we created above.

In [ ]:
label_ndjson = []
annotations = [
    radio_annotation_ndjson,
    nested_radio_annotation_ndjson,
    nested_checklist_annotation_ndjson,
    checklist_annotation_ndjson,
    text_annotation_ndjson,
    bbox_annotation_ndjson,
    bbox_with_radio_subclass_ndjson,
    polygon_annotation_ndjson,
    mask_annotation_ndjson,
    mask_with_text_subclass_ndjson,
    point_annotation_ndjson,
    polyline_annotation_ndjson,
    bbox_source_ndjson,
    bbox_target_ndjson,
    relationship_ndjson,  ## Only supported for MAL imports
]
for annotation in annotations:
    annotation.update({
        "dataRow": {
            "globalKey": global_key
        }
    })
    label_ndjson.append(annotation)

## Step 6: Upload annotations to a project as pre-labels or ground truth
For the purpose of this tutorial only import one of the annotations payloads at the time (NDJSON or Python annotation types).

Option A: Upload to a labeling project as pre-labels (MAL)

In [ ]:
# upload MAL labels for this data row in project
upload_job = lb.MALPredictionImport.create_from_objects(
    client=client,
    project_id=project.uid,
    name="mal_job" + str(uuid.uuid4()),
    predictions=label
)
upload_job.wait_until_done()

print(f"Errors: {upload_job.errors}")
print(f"Status of uploads: {upload_job.statuses}")

Option B: Upload to a labeling project using ground truth

In [ ]:
# Relationships are not supported with LabelImport
# For this demo either run MAL or Ground Truth, not both

# Upload label for this data row in project
# upload_job = lb.LabelImport.create_from_objects(
#     client = client,
#     project_id = project.uid,
#     name="label_import_job"+str(uuid.uuid4()),
#     labels=label)

# print("Errors:", upload_job.errors)
# print("Status of uploads: ", upload_job.statuses)

In [ ]:
# project.delete()
# dataset.delete()